<a href="https://colab.research.google.com/github/xqgh76/curs-git-i-github/blob/main/Miniproject_taxis_TODO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anàlisi exploratòria de dades i preprocessament

L'**anàlisi exploratòria de dades** és el tractament estadístic al qual se sotmeten les mostres recollides durant un procés de recerca en qualsevol camp científic.


**Procés bàsic**
- *Transformar les dades*: Ens serveix per saber què fer front a valors nuls, faltants o dades atípiques. A més d'establir si hi ha necessitat de reduir la dimensionalitat de dades.
- *Visualitzar*: Utilitzar alguna eina per fer una representació gràfica de les dades, per exemple, R, Jupyter notebook, Google Colab, etc.
- *Analitzar i interpretar*: Analitzar i interpretar les dades a través de diferents visualitzacions.
- *Documentar*: Documentar totes les gràfiques i estadístiques generades.

Si realitzem correctament els passos ens facilitarem la manera d'abordar aquestes dades sense deixar de banda l'objectiu o el propòsit per al qual les necessitem.

L'**objectiu d'aquesta pràctica** és aprendre com fer una explicació de dades. En aquest cas utilitzarem les dades dels **taxis grocs de la ciutat de Nova York**.

En finalitzar el notebook, haureu de ser capaços de respondre la pregunta següent:


## ¿Com ha afectat la covid a l'ús dels taxis a Nova York?

Algunes de les preguntes que ens farem al llarg del notebook són:
- Com ha canviat la covid l'ús dels taxis a NYC?
- Quina distribució d'encàrrecs segueixen els taxis i quina distància / durada tenen?
- Quines són les zones on més / menys taxis s'agafen? I a on més va la gent?
- Quins horaris són els més usuals?
- Quins dies de la setmana i del mes s'utilitzen més? Possibles motius?

In [ ]:
# adjust the width of the output container in the notebook
from IPython.core.display import display,HTML
display(HTML('<style>.container {width: 99% !important;}</style>'))

**Instal·lació i importació de les llibreries necessàries**

In [ ]:
! pip install pyarrow
! pip install pyshp
! pip install shapely

In [ ]:
# Importació de les llibreries
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
from tqdm.notebook import tqdm
import pyarrow.parquet as pq

In [ ]:
import shapefile
from shapely.geometry import Polygon
# from descartes.patch import PolygonPatch
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_theme(color_codes=False)

In [ ]:
# Variables globals
YEARS = [2019, 2020, 2021]

Primer de tot cal descarregar les dades:

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
# Download the Trip Record Data
for year in tqdm(YEARS):
    if not os.path.exists(f'data/{year}'):
        os.makedirs(f'data/{year}', exist_ok=True)
        for month in tqdm(range(1, 13)):
            urllib.request.urlretrieve(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet', f'data/{year}/{month:02d}.parquet')

## 01. Neteja de dades

Per tal de tenir unes dades netes i útils, cal eliminar totes aquelles files que continguin informació corrupta:
- La recollida és després que la finalització del trajecte.
- Les dades s'importen per mes i any. Les dates són correctes?
- Viatges amb 0 passatges?
- Viatges molts llargs o molt curts?
- Pagaments negatius.

**Observacions:**

- Per agilitzar els càlculs i reduir el temps de còmput, feu un sampleig uniforme de les dades.
- Columnes de dates són to_datetime series

In [ ]:
def load_table(year, month):
    """
    Funció que llegeix les dades descarregades i les converteix a un DataFrame

    Args:
        year (int): The year of the data to load.
        month (int): The month of the data to load.

    Returns:
        pd.DataFrame: A pandas DataFrame containing the loaded data.
    """
    return pq.read_table(f'data/{year}/{str(month).zfill(2)}.parquet').to_pandas()

In [ ]:
# TODO: Visualitza les dades d'un mes


**Data Dictionary**

De totes les columnes que tenen les dades, només calen les següents:

- *tpep_pickup_datetime*: The date and time when the meter was engaged.
- *tpep_dropoff_datetime*: The date and time when the meter was disengaged.
- *Passenger_count*: The number of passengers in the vehicle. (This is a driver-entered value)
- *Trip_distance*: The elapsed trip distance in miles reported by the taximeter.
- *PULocationID*: TLC Taxi Zone in which the taximeter was engaged
- *DOLocationID*: TLC Taxi Zone in which the taximeter was disengaged
- *Payment_type*: A numeric code signifying how the passenger paid for the trip.
    - 1= Credit card
    - 2= Cash
    - 3= No charge
    - 4= Dispute
    - 5= Unknown
    - 6= Voided trip
- *Fare_amount*: The time-and-distance fare calculated by the meter.
- *Total_amount*: The total amount charged to passengers. Does not include cash tips.

### Exercici 1

Completa el codi de la funcio  funció `clean_data` seguint els passos indicats per
netejar i filtrar el DataFrame `data`.

**La funció ha de realitzar les següents operacions de neteja i filtratge:**

1.  **Selecció de Columnes:** Conservar només les columnes especificades a `required_data`.
2.  **Mostreig de Dades:** Aplicar un mostreig basat en el paràmetre `sampling`.
3.  **Gestió de Valors Nuls i Duplicats:** Eliminar files amb valors nuls i duplicats.
4.  **Filtratge Temporal:** Assegurar que la data de baixada sigui posterior a la de recollida.
    A més, filtrar les dades per l'any i el mes especificats.
5.  **Validació de Valors Numèrics:** Eliminar registres amb valors no vàlids (negatius o zero)
    per a `passenger_count`, `trip_distance`, `fare_amount` i `total_amount`.

---

### Pistes

* **Selecció de columnes:** Pots crear una llista de columnes a eliminar i fer servir `data.drop()`.
* **Mostreig:** Utilitza `data[::sampling]` per a mostreig enter, i `data.sample(frac=sampling)` per a mostreig flotant.
* **NaNs i duplicats:** Fes servir `dropna()` i `drop_duplicates()` amb `inplace=True`.
* **Filtratge per temps i data:** Compara columnes de temps directament. Per filtrar per any i mes,
    accedeix a `.dt.year` i `.dt.month` de les columnes de datetime.
* **Validació de valors:** Utilitza condicions booleanes dins de claudàtors `data[...]` per filtrar files.

In [ ]:
required_data = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
                 'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'total_amount']

def clean_data(data, year, month, sampling = 1000):
    """
    Neteja i filtra un DataFrame de dades de trajectes de taxi.

    Args:
        data (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.
        year (int): L'any de les dades a netejar.
        month (int): El mes de les dades a netejar.
        sampling (int or float, optional): Estratègia de mostreig. Si és un 'int',
                                           pren cada 'sampling' fila. Si és un 'float'
                                           (entre 0 i 1), pren una fracció de les dades.
                                           Per defecte és 1000.
    Returns:
        pd.DataFrame: The cleaned and filtered DataFrame.

    Comprovarem:
      - tpep_dropoff_datetime - tpep_pickup_datetime > 1 && < 1 day?
      - Passenger_count > 0
      - Trip_distance > 0
      - Payment_type: A numeric code signifying how the passenger paid for the trip.
            1= Targeta de crèdit
            2= Efectiu
            3= Sense càrrec
            4= Disputa
            5= Desconegut
            6= Viatge anul·lat
      - Fare_amount > 0
      - Total_amount > 0
    """

    # TODO: Implementa el codi de la funció
    raise NotImplementedError

    return (data)

A la funció *post_processing* podeu afegir tota la informació que necessiteu sobre les dades per tal de dur a terme l'exploració necessària.

In [ ]:
def post_processing(data):
    """
    Funció on implementar qualsevol tipus de postprocessament necessari.

    Args:
        data (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.

    Returns:
        pd.DataFrame: The DataFrame with added post-processed columns.
    """

    # Passem la distància a quilòmetres
    data['trip_distance'] = data['trip_distance'] * 1.6

    # Creem el diccionari de dades amb les noves dades que volem introduir
    new_data = {'pickup_hour': data['tpep_pickup_datetime'].dt.hour,
                'dropoff_hour': data['tpep_dropoff_datetime'].dt.hour,
                'pickup_day': data['tpep_pickup_datetime'].dt.dayofweek,
                'dropoff_day': data['tpep_dropoff_datetime'].dt.dayofweek,
                'pickup_week': data['tpep_pickup_datetime'].dt.isocalendar().week,
                'dropoff_week': data['tpep_dropoff_datetime'].dt.isocalendar().week,
                'pickup_month': data['tpep_pickup_datetime'].dt.month,
                'dropoff_month': data['tpep_dropoff_datetime'].dt.month,
                'pickup_year': data['tpep_pickup_datetime'].dt.year,
                'dropoff_year': data['tpep_dropoff_datetime'].dt.year,
                'pickup_dayofyear': (data['tpep_pickup_datetime'].dt.dayofyear),
                'dropoff_dayofyear': (data['tpep_dropoff_datetime'].dt.dayofyear),
                'trip_duration': round(((data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']) / pd.Timedelta(hours=1)), 3)} # Changed np.timedelta64 to pd.Timedelta for clarity and common usage with pandas

    # I els afegim al dataframe en qüestió
    for k, v in new_data.items():
        data[k] = v

    # A continuació, creem i afegim la columna de velocitat a partir de les dades ja afegides.
    # Només contemplem els valors que tinguin un trajecte mínim de 3 minuts. Altrament, apareixen valors 'inf' a causa del temps reduït.
    data['speed'] = data['trip_distance'] / data.loc[data['trip_duration'] > 0.05, 'trip_duration']

    return data

Creem un nou dataset que contingui tota la informació dels anys: 2019, 2020, 2021.

Recordeu que per tal de reduir la memòria necessària, agafem un subsample de dades.

In [ ]:
df = pd.concat([post_processing(clean_data(load_table(year, month), year, month, 1000)) for year in tqdm(YEARS, leave = False) for month in tqdm(range(1, 13), leave = False)])

In [ ]:
df.head(10)

## 02. Visualització per anys (Gràfic de Barres)


Podem respondre directament la pregunta: **¿Ha incrementat / disminuït la covid la quantitat de viatges fets amb taxis?**

Per respondre aquesta pregunta pots crear un gràfic de barres on es visualitzin la quantitat de viatges per any.




### Exercici 2

Completar la funció `bar_plot` per tal que generi un gràfic de barres.
Aquest gràfic ha de representar el **nombre de registres (o viatges)** per cada
`pickup_year` (any de recollida). Per a fer el recompte, podeu utilitzar qualsevol
columna no nul·la, per exemple, `passenger_count`.

**La funció ha de seguir aquests passos i assignar els resultats a noves variables per a claredat:**

1.  **Crear una figura i un eix:** Inicialitza la figura i l'eix per al gràfic.
2.  **Agrupar les dades i comptar:** Agrupa el DataFrame `df` per `pickup_year` i
    compta el nombre de registres per a la `column` especificada (o qualsevol columna
    adequada per al recompte, com ara `passenger_count`). Assigna el resultat a una variable
    (p. ex., `yearly_counts`).
3.  **Generar el gràfic de barres:** Utilitza el mètode `plot.bar()` sobre les dades
    agregades per crear el gràfic, passant els arguments `xlabel`, `ylabel` i `title`.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def bar_plot(df, column, xlabel, ylabel, title):
    """
    Genera un gràfic de barres per visualitzar la quantitat de viatges per any.

    Args:
        df (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.
        column (str): The column to count for each year (e.g., 'passenger_count').
        xlabel (str): Etiqueta per a l'eix X del gràfic.
        ylabel (str): Etiqueta per a l'eix Y del gràfic.
        title (str): Títol del gràfic.
    """

    fig, ax = plt.subplots(figsize=(15,7))

    # TODO Agrupem per l'any de recollida i comptem els registres i generem gràfic de barres
    # Pista: Utilitza .groupby() amb 'pickup_year' i després .count() sobre la columna especifica

    raise NotImplementedError


In [ ]:
# TODO: Visualitza el gràfic de barres

**Pregunta: És el comportament que esperàveu? Per què?**



### Exercici 3

Ara, visualitzarem **quants passatgers hi ha per taxi i per any**.


Implementa la funció `passengers_taxi_year` per tal que generi gràfics de barres
que mostrin el nombre de viatges segons la quantitat de passatgers (`passenger_count`)
per cada any (`pickup_year`). La funció ha de ser capaç de generar tant gràfics
amb els valors absoluts com gràfics normalitzats (percentatges).

**La funció ha de seguir aquests passos i assignar els resultats a noves variables per a claredat:**

1.  **Crear una figura amb subgràfics:** Per a cada any, es vol un gràfic separat que
    mostri la distribució de passatgers.
2.  **Agrupar les dades i comptar:** Cal agrupar el DataFrame `df` per `passenger_count`
    i `pickup_year` per obtenir el recompte de viatges per cada combinació. Assigna el resultat
    a una variable (p. ex., `counts_grouped`).
3.  **Desapilar les dades:** Transforma les dades agrupades per tenir els anys com a columnes.
    Assigna el resultat a una variable (p. ex., `counts_unstacked`).
4.  **Gestionar la normalització:**
    * Si el paràmetre `norm` és `True`, normalitza les dades desagrupades de manera que
        la suma de les barres per a cada any sigui 1 (o 100%). Assigna el resultat a una variable
        (p. ex., `normalized_data`).
    * Si `norm` és `False` (valor per defecte), utilitza les dades desagrupades directament.
5.  **Generar els gràfics de barres:** Utilitzar `plot.bar()` sobre el DataFrame final per
    visualitzar les dades, passant els arguments `xlabel`, `ylabel`, `ylim`, `title` i `subplots=True`.

---

### Pistes per completar el codi

* **Agrupació i recompte:** Utilitza `.groupby(['col1', 'col2'])['col_to_count'].count()`.
* **Desapilar:** Aplica `.unstack()` al resultat de l'agrupació per pivotar els anys a columnes.
* **Normalització:** Per normalitzar cada columna (any) de forma independent, pots fer
    `df_unstacked.transform(lambda x: x / x.sum())`.


In [ ]:
def passengers_taxi_year(df, ylim, xlabel, ylabel, title, norm = False):
    """
    Args:
        df (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.
        ylim (tuple): Limits per a l'eix Y del gràfic.
        xlabel (str): Etiqueta per a l'eix X del gràfic.
        ylabel (str): Etiqueta per a l'eix Y del gràfic.
        title (str): Títol del gràfic.
        norm (bool, optional): Si és True, normalitza els resultats a percentatges. Per defecte és False.
    """
    # TODO
    # Pas 1: Agrupem les dades per passatgers i any, i comptem els viatges
    # Pista: Utilitza .groupby() i .count().

    # Pas 2: Desapilem les dades per tenir els anys com a columnes
    # Pista: Aplica .unstack() al resultat de l'agrupació.

    # En cas que es vulgui normalitzat
    # if norm:
        # Pas 3: Normalitzem les dades desagrupades per tenir percentatges
        # Pista: Utilitza .transform(lambda x: x / x.sum()) sobre counts_unstacked.
        # Pas 4: Grafiquem el resultat normalitzat

    # En cas que NO es vulgui normalitzat
    # else:
        # Pas 3b: Si no normalitzem, les dades a graficar són les mateixes que les desagrupades
        # Pas 4: Grafiquem el resultat (comptes absoluts)

In [ ]:
# TODO: Crida la funció per visualitzar el recompte de passatgers per any

In [ ]:
# TODO: Crida la funció per visualitzar el recompte de passatgers per any en percentatge

A la figura anterior hem visualitzat cada any per separat. Per tal que la visualització sigui més ràpida i simple d'interpretar, unim tota la informació en un gràfic.

In [ ]:
def passengers_taxi(df, xlabel, ylabel, norm = False):
    """
    Funció que visualitza quants passatgers hi ha per taxi

    Args:
        df (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.
        xlabel (str): Etiqueta per a l'eix X del gràfic.
        ylabel (str): Etiqueta per a l'eix Y del gràfic.
        norm (bool, optional): Si és True, normalitza els resultats a percentatges. Per defecte és False.
    """
    fig, ax = plt.subplots(figsize=(15,8))

    if norm:
        df.groupby(['passenger_count','pickup_year'])['pickup_year'].count().unstack().transform(lambda x: x/x.sum()).plot.bar(ax=ax, xlabel=xlabel, ylabel=ylabel, subplots=False, grid=False)

    else:
        df.groupby(['passenger_count','pickup_year'])['pickup_year'].count().unstack().plot.bar(ax=ax, xlabel=xlabel, ylabel=ylabel, subplots=False, grid=False)

    fig.suptitle("Passatgers per taxi")

In [ ]:
# TODO: Crida la funció per visualitzar el recompte de passatgers per any

In [ ]:
# TODO: Crida la funció per visualitzar el recompte de passatgers per any  en percentatge

**Pregunta: Quin impacte heu vist sobre les dades? Creieu que la covid va tenir molt impacte?**

## 03. Quantitat de viatges

Fins ara hem vist la quantitat de viatges que hi ha hagut en els anys estudiats.

Anem a estudiar quins canvis es poden veure si agreguem les dades per hores, dies de la setmana, setmana de l'any i mes.

Aquests gràfics han de ser de línies discontínues i marcar amb una rodona o creu allà on està el valor

In [ ]:
def visualize_trips(df, columns, title, xlabel, ylabel):
    """
    Funció que visualitza els viatges per diferents agregacions de dades
    :param df: DataFrame amb les dades dels viatges.
    :param column: Columna on estan les dades d'interès
    :param title: Titol de la figura
    :param xlabel: Etiqueta de l'eix X
    :param ylabel: Etiqueta de l'eix Y
    """
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    fig, axs = plt.subplots(1, 2, figsize = (15, 3), sharey = True)
    # Per cada columna en column_data
    for column, ax in zip(columns, axs):
      # Per cada any a Years
      for year in YEARS:
        # Filtrar les dades per column i any
        counts = df.loc[df.pickup_year == year, column].value_counts().sort_index()

        # Reordenem en el cas de dies de la setmana
        if set(counts.index).issubset(set(days_order)):
          counts = counts.reindex(days_order)

        # Fer el 'plt.plot' i el 'plt.scatter'
        ax.plot(counts.index, counts.values)
        ax.scatter(counts.index, counts.values, marker = 'x', label = year)
        ax.legend()
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(column)

    plt.suptitle(title)
    plt.show()

In [ ]:
visualize_trips(df, ['pickup_hour', 'dropoff_hour'], title = 'Quantitat de viatges per hora', xlabel = 'Hora del dia', ylabel = 'Quanitat')

In [ ]:
visualize_trips(df, ['pickup_day', 'dropoff_day'], title = 'Quantitat de viatges per dia de la setmana', xlabel = 'Dia de la setmana', ylabel = 'Quanitat')

In [ ]:
visualize_trips(df, ['pickup_week', 'dropoff_week'], title = 'Quantitat de viatges per setmana de l\'any', xlabel = 'Setmana de l\'any', ylabel = 'Quanitat')

In [ ]:
visualize_trips(df,['pickup_month', 'dropoff_month'], title = 'Quantitat de viatges per mes', xlabel = 'Mes de l\'any', ylabel = 'Quanitat')

**Pregunta: Quins comportaments veieu en cada cas? Quin creieu que és el motiu?**

## 04. Relació distancia - temps i velocitat

A les dades tenim la distància que ha recorregut el taxi en cada viatge i de la informació temporal podem extreure també la duració d'aquest.


Intentarem esbrinar com la covid va afectar les distàncies i les durades dels viatges juntament amb la velocitat dels taxis.

Creieu que la densitat de trànsit va variar?

### Histogrames

Per començar visualitza els **histogrames** de distància i durada per any.

Pots fer servir la funció *plt.hist()* o *plt.bar()*.

In [ ]:
def visualize_histograms(df, column, title, xlabel, ylabel, xlim):
    """
    Funció que crea un histograma a partir de la informació que conté la columna del dataframe

    Args:
        df (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.
        column (str): The column to create a histogram from.
        title (str): Títol de l'histograma.
        xlabel (str): Etiqueta per a l'eix X de l'histograma.
        ylabel (str): Etiqueta per a l'eix Y de l'histograma.
        xlim (tuple): Limits per a l'eix X de l'histograma.
    """
    # TODO: Implement histogram
    raise NotImplementedError

In [ ]:
visualize_histograms(df, 'trip_distance', title = 'Distancia dels viatge per any',
                     xlabel = 'Distancia (km)', ylabel = 'Quanitat', xlim = (-5, 20))

visualize_histograms(df, 'trip_duration', title = 'Durada dels viatge per any', xlabel = 'Duració (h)', ylabel = 'Quanitat', xlim = (-1, 5))

**PREGUNTES:**

* Com creieu que la covid va afectar les distàncies i durades dels viatges?

* I a la velocitat dels taxis?

### Gràfic de dispersió i correlació

Crea gràfics de dispersió per il·lustrar la relació entre la durada i la distància dels viatges.

Es possible que les dades continguin mostres fora la distribució (outliers). En aquest cas, omet aquestes dades i torna a visualitzar el grafic.

Per veure si existeix alguna correlació, es interesant que utilitzeu la funció *sns.regplot()*.

In [ ]:
def scatter_plot(df, x_value, y_value, xlabel, ylabel, remove_outliers = False):
    """
    Funció que mostra un scatter plot donades dues dades
    :param df: DataFrame amb les dades dels viatges.
    :param x_value: Nom de la columna on estan els valors
    :param y_value: Nom de la columna on estan els valors
    :param xlabel: Etiqueta de l'eix X
    :param ylabel: Etiqueta de l'eix Y
    :param remove_outliers:
    """

    def remove(df, column):
      # Suposem que es volen eliminar outliers més enllà de 2 desviacions estàndard per la columna d'interès
      mean_val = df[column].mean()
      std_val = df[column].std()
      df = df[(df[column] > mean_val - 2 * std_val) & (df[column] < mean_val + 2 * std_val)]
      return df

    # Eliminar outliers
    if remove_outliers:
      df = remove(df, x_value)
      df = remove(df, y_value)

    fig, axs = plt.subplots(1, len(YEARS), figsize = (18, 3), sharey=True)

    # Per cada any
    for year, ax in zip(YEARS, axs):
      # Filter
      df_year = df[df.pickup_year == year]
      # Scatter
      ax.scatter(df_year.trip_distance, df_year.trip_duration, alpha = 0.5)
      # Add regression
      sns.regplot(x = df_year.trip_distance, y = df_year.trip_duration, ax = ax, color='r',scatter=False,)

    plt.show()


In [ ]:
scatter_plot(df, 'trip_distance', 'trip_duration', 'Distancia (km)', 'Temps (h)')

In [ ]:
scatter_plot(df, 'trip_distance', 'trip_duration', 'Distancia (km)', 'Temps (h)', True)

**Pregunta: Pots veure alguna relació? Pots calcular la correlació entre les dades per treure més informació?**

Tal com fèiem a l'apartat 3, visualitzeu les dades temporals i de distància a partir de les setmanes, i mesos de l'any.

In [ ]:
def visualize_per_period(df, column_data, columns, xlabel, ylabel, title):
    """
    Funció que visualitza la distància / duració dels trajectes en un temps a determinar
    :param df: DataFrame amb les dades dels viatges.
    :param column_data: Nom de la columna on estan els valors
    :param columns: Nom de les columnes (pickup, dropoff)
    :param xlabel: Etiqueta de l'eix X
    :param ylabel: Etiqueta de l'eix Y
    :param title: Títol de la figura
    """

    fig, axs = plt.subplots(1, len(columns), figsize = (15, 3), sharey=True)
    # Per cada columna de column_data
    for column, ax in zip(columns, axs):

      # Utilitzem groupby + unstack  o  pivot_table
      # OPCIO 1
      group = df.groupby(['pickup_year', column]).mean(numeric_only = True)[column_data].unstack()

      # OPCIO 2
      # group = df.pivot_table(index = 'year', columns = 'pickup_week', values = 'trip_distance', aggfunc = 'mean')

      # per cada any
      for year in YEARS:
        # filtratge + plot + scatter
        ax.scatter(group.loc[year].index, group.loc[year].values, label = year)
        # group.loc[group.index == year].index
        ax.plot(group.loc[year].index, group.loc[year].values, alpha = 0.5)
        ax.legend()
      ax.set_title(column)
      ax.set_xlabel(xlabel)
      ax.set_ylabel(ylabel)

In [ ]:
visualize_per_period(df, 'trip_distance', columns = ['pickup_week', 'dropoff_week'],
                    xlabel = 'Setmana de l\'any', ylabel = 'Distancia mitjana (km)', title = 'Distancia dels viatges per setmanes')

In [ ]:
visualize_per_period(df, 'trip_distance', columns = ['pickup_month', 'dropoff_month'],
                     xlabel = 'Mes de l\'any', ylabel = 'Distancia mitjana (km)', title = 'Distancia dels viatges per mesos')

In [ ]:
visualize_per_period(df, 'trip_duration', columns = ['pickup_week', 'dropoff_week'],
                     xlabel = 'Setmana de l\'any', ylabel = 'Durada mitjana (h)', title = 'Durada dels viatges per setmanes')

In [ ]:
visualize_per_period(df, 'trip_duration', columns = ['pickup_month', 'dropoff_month'],
                     xlabel = 'Mes de l\'any', ylabel = 'Durada mitjana (h)', title = 'Durada dels viatges per mesos')

**Pregunta: Hi ha algun comportament estrany a part de la covid? Per què pot ser causat?**
  

Fins ara hem mostrat les dades de manera agregada.

Per tal de visualitzar-ho de manera global, utilitzarem la funció *plt.imshow()* que visualitza imatges i, per tant, matrius.

Implementa una funció que visualitzi per any:

- un mapa de calor que indiqui a quina hora del dia hi ha viatges més llargs durant l'any.
- un mapa de calor que indiqui a quina hora del dia hi ha viatges més llargs durant la setmana.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def heatmap(df, group, column_data, xlabel, ylabel, columns=None):
    """
    Funció que agrega les dades de manera adient per visualitzar un mapa de calor.

    Args:
        df (pd.DataFrame): El DataFrame d'entrada amb les dades dels trajectes de taxi.
        group (list): Una llista de noms de columnes per agrupar per a l'agregació.
        column_data (str): La columna que conté les dades a agregar (per exemple, 'trip_duration').
        xlabel (str): Etiqueta per a l'eix X del mapa de calor.
        ylabel (str): Etiqueta per a l'eix Y del mapa de calor.
        columns (list, optional): Una llista d'etiquetes per a les marques de l'eix Y. Per defecte és None.
    """
    # Agreguem les dades per any i el grup especificat, i desagrupem els anys a columnes
    aggregated_data = df.groupby(['pickup_year'] + group)[column_data].sum().unstack(level=0)

    # Calculem els valors mínim i màxim per a la barra de color
    vmin = aggregated_data.min().min()
    vmax = aggregated_data.max().max()

    # Determinem els anys a plotar (assumint 2019, 2020, 2021 com els més probables)
    years_to_plot = sorted([year for year in df['pickup_year'].unique() if year >= 2019])[:3]

    # Ajustem l'alçada de la figura si hi ha moltes categories a l'eix Y
    fig_height = 6 if columns is None or len(columns) <= 10 else 18
    fig, ax = plt.subplots(nrows=len(years_to_plot), ncols=1, figsize=(20, fig_height), layout='constrained')

    # Assegurem que 'ax' sigui sempre una llista per facilitar la iteració
    if len(years_to_plot) == 1:
        ax = [ax]

    # Iterem sobre cada any per crear un subplot
    for i, year in enumerate(years_to_plot):
        current_year_data = aggregated_data[year]

        # Preparem les dades per a imshow, transposant si cal per l'eix Y
        plot_data = current_year_data.unstack().T if columns is not None else current_year_data.unstack()
        im = ax[i].imshow(plot_data, vmin=vmin, vmax=vmax, interpolation='nearest', cmap='jet', aspect='auto')

        # Configurem títols i etiquetes
        ax[i].set_title(str(year))
        ax[i].set_xlabel(xlabel)
        ax[i].set_ylabel(ylabel)
        ax[i].grid(False)

        # Si s'especifiquen columnes, assignem les marques i etiquetes de l'eix Y
        if columns is not None:
            ax[i].set_yticks(range(len(columns)))
            ax[i].set_yticklabels(columns)

    # Afegim una barra de color global
    fig.colorbar(im, ax=ax, label="Suma de: " + column_data, shrink=0.7)

    plt.show()

In [ ]:
heatmap(df, ['pickup_hour', 'pickup_dayofyear'], 'trip_duration', 'Dies de l\'any', 'Hores del dia')

In [ ]:
heatmap(df, ['pickup_hour', 'pickup_day'], 'trip_duration', 'Hores del dia', 'Dies de la setmana', ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

Repeteix els gràfics anteriors visualitzant la distancia.

In [ ]:
heatmap(df, ['pickup_hour', 'pickup_dayofyear'], 'trip_distance', 'Dies de l\'any', 'Hores del dia')

In [ ]:
heatmap(df, ['pickup_hour', 'pickup_day'], 'trip_distance', 'Hores del dia', 'Dies de la setmana', ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

**Pregunta: Quines conclusions treieu dels mapes de calor?**


## 05. Visualitzem les localitzacions dels viatges

### Funcions d'ajuda per visualitzar dades geogràfiques

In [ ]:
# Helper functions for visualization
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import matplotlib as mpl

def get_lat_lon(gdf, col_id):
    """
    Calcula la latitud i longitud dels centroides de les geometries en un GeoDataFrame.

    Args:
        gdf (geopandas.GeoDataFrame): El GeoDataFrame d'entrada amb les geometries.
        col_id (str): El nom de la columna que conté els identificadors de localització.

    Returns:
        pd.DataFrame: Un DataFrame amb els identificadors de localització, longituds i latituds dels centroides.
    """
    content = []
    for _, row in gdf.iterrows():
        centroid = row.geometry.centroid
        content.append((row[col_id], centroid.x, centroid.y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

In [ ]:
def get_boundaries(gdf):
    """
    Calcula els límits totals d'un GeoDataFrame.

    Args:
        gdf (geopandas.GeoDataFrame): El GeoDataFrame d'entrada.

    Returns:
        tuple: Una tupla (lat_min, lat_max, lon_min, lon_max) representant els límits.
    """
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    margin = 0.01
    lat_min, lon_min, lat_max, lon_max = bounds[0] - margin, bounds[1] - margin, bounds[2] + margin, bounds[3] + margin
    return lat_min, lat_max, lon_min, lon_max

In [ ]:
def draw_zone_map(ax, gdf, col_id, col_zone, col_area, heat={}, text=[], arrows=[]):
    """
    Dibuixa un mapa de zones de taxi amb opcions per a mapa de calor, text i fletxes.

    Args:
        ax (matplotlib.axes.Axes): L'eix de matplotlib on dibuixar el mapa.
        gdf (geopandas.GeoDataFrame): El GeoDataFrame amb les geometries de les zones de taxi.
        col_id (str): El nom de la columna que conté els identificadors de localització.
        col_zone (str): El nom de la columna que conté els noms de les zones.
        col_area (str): El nom de la columna que conté l'àrea de les geometries.
        heat (dict, optional): Un diccionari on les claus són LocationIDs i els valors són les dades per al mapa de calor. Per defecte és un diccionari buit.
        text (list, optional): Una llista de LocationIDs per etiquetar amb text. Per defecte és una llista buida.
        arrows (list, optional): Una llista de diccionaris que descriuen les fletxes a dibuixar. Per defecte és una llista buida.
    """
    continent = [235/256, 151/256, 78/256]
    ocean = (89/256, 171/256, 227/256)
    theta = np.linspace(0, 2*np.pi, len(text)+1).tolist()
    ax.set_facecolor(ocean)

    # Plot the base map or heatmap using gdf.plot
    if heat:
        # Create a temporary column for merging heat data
        heat_df = pd.DataFrame(list(heat.items()), columns=[col_id, 'heat_value'])
        merged_gdf = gdf.merge(heat_df, on=col_id, how='left')
        merged_gdf['heat_value'] = merged_gdf['heat_value'].fillna(0) # Fill non-matching zones with 0 heat

        norm = mpl.colors.Normalize(vmin=merged_gdf['heat_value'].min(), vmax=merged_gdf['heat_value'].max())
        cmap = plt.get_cmap('Reds')
        merged_gdf.plot(column='heat_value', ax=ax, cmap=cmap, edgecolor='black', linewidth=0.5, legend=False)

        # Add color bar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        plt.colorbar(sm, ax=ax, orientation='vertical', label="Heat Value") # Label for the color bar
    else:
        gdf.plot(ax=ax, facecolor=continent, edgecolor='black', linewidth=0.5)

    # Handle text labels and arrows
    if text:
        df_text = get_lat_lon(gdf[gdf[col_id].isin(text)], col_id)
        for _, row in df_text.iterrows():
            loc_id = row[col_id]
            zone = gdf[gdf[col_id] == loc_id][col_zone].iloc[0]
            x, y = row['longitude'], row['latitude']

            eta_x = 0.05 * np.cos(theta[text.index(loc_id)])
            eta_y = 0.05 * np.sin(theta[text.index(loc_id)])
            ax.annotate(f"[{loc_id}] {zone}", xy=(x, y), xytext=(x + eta_x, y + eta_y),
                        bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12,
                        arrowprops=dict(facecolor='black', width=3, shrink=0.05))
    else:
        # If no specific text labels, label larger zones
        df_label = get_lat_lon(gdf[gdf[col_area] > 0.0001], col_id)
        for _, row in df_label.iterrows():
             ax.text(row['longitude'], row['latitude'], str(row[col_id]), ha='center', va='center')


    for arr in arrows:
        ax.annotate('', xy=arr['dest'], xytext=arr['src'], size=arr['cnt'],
                    arrowprops=dict(arrowstyle="fancy", fc="0.6", ec="none"))

    lat_min, lat_max, lon_min, lon_max = get_boundaries(gdf)
    ax.set_xlim(lat_min, lat_max)
    ax.set_ylim(lon_min, lon_max)

In [ ]:
def draw_region_map(ax, gdf, col_borough, heat={}):
    """
    Dibuixa un mapa de districtes (boroughs) amb opció per a mapa de calor.

    Args:
        ax (matplotlib.axes.Axes): L'eix de matplotlib on dibuixar el mapa.
        gdf (geopandas.GeoDataFrame): El GeoDataFrame amb les geometries dels districtes.
        col_borough (str): El nom de la columna que conté els noms dels districtes.
        heat (dict, optional): Un diccionari on les claus són els noms dels districtes i els valors són les dades per al mapa de calor. Per defecte és un diccionari buit.
    """
    continent = [235/256, 151/256, 78/256]
    ocean = (89/256, 171/256, 227/256)
    ax.set_facecolor(ocean)

    reg_list = {'Staten Island':1, 'Queens':2, 'Bronx':3, 'Manhattan':4, 'EWR':5, 'Brooklyn':6}
    reg_centers = {k: [] for k in reg_list}

    if heat:
        norm = mpl.colors.Normalize(vmin=math.sqrt(min(heat.values())), vmax=math.sqrt(max(heat.values())))
        cmap = plt.get_cmap('Reds')
    else:
        norm = mpl.colors.Normalize(vmin=1, vmax=6)
        cmap = plt.get_cmap('Pastel1')

    for _, row in gdf.iterrows():
        borough = row[col_borough]
        geom = row.geometry

        if heat:
            value = math.sqrt(heat[borough])
            color = cmap(norm(value))[:3]
        else:
            color = cmap(norm(reg_list[borough]))[:3]

        gpd.GeoSeries([geom]).plot(ax=ax, facecolor=color, edgecolor='black', linewidth=0.5)

        centroid = geom.centroid
        reg_centers[borough].append((centroid.x, centroid.y))

    for k in reg_list:
        x_coords, y_coords = zip(*reg_centers[k])
        x, y = np.mean(x_coords), np.mean(y_coords)
        label = f"{k}\n({heat[k]/1000:.0f}K)" if heat else k
        ax.text(x, y, label, ha='center', va='center',
                bbox=dict(facecolor='black', alpha=0.5), color='white', fontsize=12)

    lat_min, lat_max, lon_min, lon_max = get_boundaries(gdf)
    ax.set_xlim(lat_min, lat_max)
    ax.set_ylim(lon_min, lon_max)

### Importem dades de geodata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the path to your shapefile
shapefile_path = './drive/MyDrive/dataset/geodata_taxis/taxi_zones.shp'

# Use geopandas to open the shapefile
sf = gpd.read_file(shapefile_path)

fields_name = sf.columns.tolist()
print("Field Names:", fields_name)

shp_attr = sf.drop(columns=['geometry']).to_dict(orient='records')
print("\nFirst 5 attributes (as dictionaries):")
for i, attr in enumerate(shp_attr[:5]):
    print(attr)


In [ ]:
sf.head()

In [ ]:
df_loc_coords = get_lat_lon(sf, 'LocationID')
df_loc_coords['LocationID'] = df_loc_coords['LocationID'].astype(int)
sf['LocationID'] = sf['LocationID'].astype(int)

df_loc = sf.join(df_loc_coords.set_index("LocationID"), on="LocationID")
df_loc.head()

In [ ]:
df.head()

Només tenim la ID d'una localització, per tant, necessitem aconseguir la latitud i la longitud.

La mateixa pàgina dels taxis ens dóna el fitxer *taxi_zones.zip*, però primer cal que convertim les dades de coordenades amb format WGS84.

Podem utilitzar aquesta web: https://mygeodata.cloud/

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20,20))
ax = plt.subplot(1, 2, 1)
plt.title("NYC Borough Map")
draw_region_map(ax, sf, col_borough='borough')
ax.grid(False)
ax = plt.subplot(1, 2, 2)
plt.title("Taxi Zone Map")
draw_zone_map(ax, sf, col_id='LocationID', col_zone='zone', col_area='Shape_Area')
plt.show()


In [ ]:
df.shape

In [ ]:
df_merged = pd.merge(df, df_loc, left_on = 'PULocationID', right_on = 'LocationID')

In [ ]:
df_merged.head(2)

In [ ]:
df_merged.shape

## 06. ¿Quines zones tenen més recollides i quines zones tenen més entregues?

En aquest apartat volem conèixer i visualitzar les zones on els taxis són més utilitzats.

El primer pas és ordenar i guardar en una variable els llocs més comuns en la recollida i el destí.

Printa els 5 llocs més freqüents per any en cada cas.

In [ ]:
# Recollida
top_pu = df_merged.groupby(['pickup_year','PULocationID'])['PULocationID'].count().reset_index(name='PUcount').sort_values(by='PUcount',ascending=False)
top_pu = pd.merge(top_pu, sf[['LocationID','zone']], left_on='PULocationID', right_on='LocationID').drop(labels='LocationID',axis=1)

# Destí
top_do = df_merged.groupby(['pickup_year','DOLocationID'])['DOLocationID'].count().reset_index(name='DOcount').sort_values(by='DOcount',ascending=False)
top_do = pd.merge(top_do, sf[['LocationID','zone']], left_on='DOLocationID', right_on='LocationID').drop(labels='LocationID',axis=1)

In [ ]:
n_top = 5
def show_top_n(df, column, df_loc, n_top = n_top):
    """
    Funció que mostra els llocs més usuals per any

    Args:
        df (pd.DataFrame): El DataFrame d'entrada amb les dades agrupades i el recompte de localitzacions.
        column (str): El nom de la columna que indica si són localitzacions de recollida ('PULocationID') o de destinació ('DOLocationID').
        df_loc (pd.DataFrame): DataFrame amb informació de les localitzacions, incloent 'LocationID' i 'zone'.
        n_top (int, optional): El nombre de localitzacions principals a mostrar. Per defecte és 5.
    """
    if column =='PULocationID':
        for i in range(3):
            print("\n\n\n\n            TOP 5 PU LOCATIONS {}".format(2019+i))
            display(top_pu.groupby(['pickup_year']).get_group(2019+i).head(n_top)[['zone','PULocationID','PUcount']])

    else:
        for i in range(3):
            print("\n\n\n\n            TOP 5 DO LOCATIONS {}".format(2019+i))
            display(top_do.groupby(['pickup_year']).get_group(2019+i).head(n_top)[['zone','DOLocationID','DOcount']])

In [ ]:
show_top_n(top_pu, 'PULocationID', sf)

**Visualitzem amb un mapa de calor quines són les zones més recurrents**

In [ ]:
for i in range(len(YEARS)):

    fig, ax = plt.subplots(1,2,figsize=(25,10))

    ax = plt.subplot(1, 2, 1)
    ax.set_title(f"Zones with most pickups - {2019+i}")
    draw_zone_map(ax, sf,'LocationID', 'zone', 'Shape_Area', heat=top_pu.set_index('PULocationID').groupby(['pickup_year']).get_group((2019+i))['PUcount'].to_dict(), text=list(top_pu.groupby(['pickup_year']).get_group((2019+i))['PULocationID'].head(5)))

    ax = plt.subplot(1, 2, 2)
    ax.set_title(f"Zones with most dropoffs - {2019+i}")
    draw_zone_map(ax, sf,'LocationID', 'zone', 'Shape_Area', heat=top_do.set_index('DOLocationID').groupby(['pickup_year']).get_group((2019+i))['DOcount'].to_dict(), text=list(top_do.groupby(['pickup_year']).get_group((2019+i))['DOLocationID'].head(5)))


**Pregunta: Per què creieu que la zona de Manhattan té més quantitat de viatges?**
